In [7]:
import os
import numpy as np
import librosa
import tensorflow as tf
from tensorflow.keras import layers, models, Input
from tensorflow.keras.callbacks import Callback
from sklearn.model_selection import train_test_split


In [8]:
# Direktori dataset
INPUT_DIR = 'dataset'

# Direktori tempat menyimpan model
MODEL_DIR = 'model'
# Buat direktori jika belum ada
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)

In [9]:
# Fungsi untuk load dan ekstrak Mel-spectrogram
def load_and_extract_spectrogram(file_path, n_mels=128, n_fft=2048, hop_length=512, max_time_steps=128):
    y, sr = librosa.load(file_path, sr=None)
    mel_spec = librosa.feature.melspectrogram(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels)
    mel_spec_db = librosa.power_to_db(mel_spec, ref=np.max)
    if mel_spec_db.shape[1] < max_time_steps:
        pad_width = max_time_steps - mel_spec_db.shape[1]
        mel_spec_db = np.pad(mel_spec_db, ((0, 0), (0, pad_width)), mode='constant')
    else:
        mel_spec_db = mel_spec_db[:, :max_time_steps]
    return mel_spec_db

In [10]:
# Loop melalui setiap folder (kelas)
subdirectories = [subdir for subdir in os.listdir(INPUT_DIR) if os.path.isdir(os.path.join(INPUT_DIR, subdir))]


In [11]:
class CustomEarlyStopping(Callback):
    def __init__(self, val_loss_threshold=0.15, val_accuracy_threshold=0.96, patience=3):
        super(CustomEarlyStopping, self).__init__()
        self.val_loss_threshold = val_loss_threshold
        self.val_accuracy_threshold = val_accuracy_threshold
        self.patience = patience
        self.best_weights = None
        self.wait = 0

    def on_epoch_end(self, epoch, logs=None):
        val_loss = logs.get('val_loss')
        val_accuracy = logs.get('val_accuracy')

        # Check if val_loss is less than the threshold and val_accuracy is greater than the threshold
        if val_loss < self.val_loss_threshold and val_accuracy > self.val_accuracy_threshold:
            print(f" Stopping training as val_loss ({val_loss:.4f}) is below {self.val_loss_threshold} and val_accuracy ({val_accuracy:.4f}) is above {self.val_accuracy_threshold * 100:.2f}%")
            self.model.stop_training = True

        # Implement patience to wait for a few more epochs before stopping
        if self.model.stop_training:
            self.wait += 1
            if self.wait >= self.patience:
                self.model.stop_training = True
        else:
            self.wait = 0  # Reset patience if improvement is seen

        # Store the best weights if this is the best result so far
        if val_accuracy > self.val_accuracy_threshold and val_loss < self.val_loss_threshold:
            self.best_weights = self.model.get_weights()

    def on_train_end(self, logs=None):
        # Restore the best weights after training ends
        if self.best_weights is not None:
            self.model.set_weights(self.best_weights)
            print("Best weights restored.")

In [12]:
for subdir in subdirectories:
    print(f"Processing class: {subdir}")
    subdir_path = os.path.join(INPUT_DIR, subdir)
    
    # Load dataset untuk kelas ini
    X = []
    y = []
    wav_files = [file for file in os.listdir(subdir_path) if file.endswith('.wav')]
    
    for wav_file in wav_files:
        file_path = os.path.join(subdir_path, wav_file)
        spectrogram = load_and_extract_spectrogram(file_path)
        X.append(spectrogram)
        y.append(1)  # Label 1 untuk kelas positif (suara benar)

    # Tambahkan data negatif (misalnya suara acak atau dari kelas lain)
    for other_subdir in subdirectories:
        if other_subdir != subdir:
            other_subdir_path = os.path.join(INPUT_DIR, other_subdir)
            other_wav_files = [file for file in os.listdir(other_subdir_path) if file.endswith('.wav')]
            for wav_file in other_wav_files:
                file_path = os.path.join(other_subdir_path, wav_file)
                spectrogram = load_and_extract_spectrogram(file_path)
                X.append(spectrogram)
                y.append(0)  # Label 0 untuk kelas negatif (suara salah)

    # Konversi ke numpy array
    X = np.array(X)
    y = np.array(y)

    # Split dataset menjadi train dan test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Tambahkan dimensi ekstra untuk CNN
    X_train = X_train[..., np.newaxis]
    X_test = X_test[..., np.newaxis]

    # Definisikan model CNN untuk klasifikasi biner
    model = models.Sequential([
        Input(shape=(X_train.shape[1], X_train.shape[2], 1)),  # Gunakan Input secara eksplisit
        layers.Conv2D(32, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(64, activation='relu'),
        layers.Dense(1, activation='sigmoid')  # Output untuk klasifikasi biner
    ])

    # Kompilasi model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Inisiasi callback CustomEarlyStopping
    # custom_early_stopping = CustomEarlyStopping(val_loss_threshold=0.15, val_accuracy_threshold=0.96, patience=3)

    # Latih model
    model.fit(X_train, y_train, epochs=8, validation_data=(X_test, y_test))

    # Simpan model ke dalam direktori 'model'
    model.save(os.path.join(MODEL_DIR, f'model_{subdir}.keras'))

    # Evaluasi model
    test_loss, test_acc = model.evaluate(X_test, y_test)
    print(f"Test accuracy for {subdir}: {test_acc * 100:.2f}%")

Processing class: 01. alif_fathah
Epoch 1/8
24/24 ━━━━━━━━━━━━━━━━━━━━ 3s 93ms/step - accuracy: 0.9218 - loss: 1.3014 - val_accuracy: 0.7784 - val_loss: 0.4546
Epoch 2/8
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step - accuracy: 0.9584 - loss: 0.1710 - val_accuracy: 0.9946 - val_loss: 0.0414
Epoch 3/8
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 97ms/step - accuracy: 0.9815 - loss: 0.0819 - val_accuracy: 0.9946 - val_loss: 0.0257
Epoch 4/8
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step - accuracy: 0.9841 - loss: 0.0620 - val_accuracy: 0.9946 - val_loss: 0.0093
Epoch 5/8
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step - accuracy: 0.9836 - loss: 0.0653 - val_accuracy: 0.9946 - val_loss: 0.0125
Epoch 6/8
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step - accuracy: 0.9886 - loss: 0.0475 - val_accuracy: 1.0000 - val_loss: 0.0070
Epoch 7/8
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 92ms/step - accuracy: 0.9881 - loss: 0.0305 - val_accuracy: 0.9946 - val_loss: 0.0097
Epoch 8/8
24/24 ━━━━━━━━━━━━━━━━━━━━ 2s 91ms/step - accuracy: 0.9865 - loss: 0.0504 